In [1]:
import sys
sys.path.append(r"C:\Users\aks14\Documents\l0bnb")

In [2]:
from scripts.generate_data import GenData

dataset = "I"
parameter = 0.9
n = 100
p = 10
supp_size = 5
snr = 1000

X, y, coefs_true, cov = GenData(dataset, parameter, n, p, supp_size, snr)

In [3]:
coefs_true

array([-0.37345297,  0.        , -0.22417852,  0.        ,  0.69262183,
        0.        ,  0.87107814,  0.        ,  0.        ,  0.33949207])

In [11]:
'''
Primal of relaxation
'''

import cvxpy as cp
import numpy as np
from itertools import chain

# lambda_0 = 0.001
# lambda_2 = 0.1
# M = 0.3

lambda_0 = 0.001
lambda_2 = 0.1
M = 0.005



n, p = X.shape

B = cp.Variable(p)
z = cp.Variable(p, boolean=True)
s = cp.Variable(p)

cons_box = [z>=0, z<=1, s>=0]
cons_bigm = [B >= -M*z, B <= M*z]
cons_conic = [z[i] >= cp.quad_over_lin(B[i], s[i]) for i in range(p)]
constraints = cons_box + cons_bigm + cons_conic

# Build the objective
obj = cp.Minimize(0.5*cp.sum_squares(X*B - y) + lambda_0*cp.sum(z) + lambda_2*cp.sum(s))
print("Built obj")

# Define the problem
prob = cp.Problem(obj, constraints)
print("Defined problem")
result = prob.solve(solver=cp.MOSEK, verbose=True, mosek_params={'MSK_DPAR_MIO_TOL_REL_GAP': 1e-2})

Built obj
Defined problem


Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 192             
  Cones                  : 11              
  Scalar variables       : 173             
  Matrix variables       : 0               
  Integer variables      : 10              

Optimizer started.
Mixed integer optimizer started.
Threads used: 4
Presolve started.
Presolve terminated. Time = 0.00
Presolved problem: 162 variables, 151 constraints, 1222 non-zeros
Presolved problem: 0 general integer, 10 binary, 152 continuous
Clique table size: 0
BRANCHES RELAXS   ACT_NDS  DEPTH    BEST_INT_OBJ         BEST_RELAX_OBJ       REL_GAP(%)  TIME  
0        1        1        0        6.3131661667e+01     6.3131050134e+01     9.69e-04    0.0   
An optimal solution satisfying the relative gap tolerance of 1.00e+00(%) has been located.
The relative gap is 9.69e-04(%).

Obje

In [49]:
prob.get_problem_data(cp.MOSEK) # ['solve_time']

({'bool_vars_idx': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
  'int_vars_idx': [],
  'c': array([0.5  , 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001,
         0.001, 0.001, 0.1  , 0.1  , 0.1  , 0.1  , 0.1  , 0.1  , 0.1  ,
         0.1  , 0.1  , 0.1  , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
         0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
         0.   , 0.   , 0.   , 0.   , 0.   ]),
  'obj_offset': 0.0,
  'dims': {'q': [102, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3],
   'ep': [],
   's': [],
   'l': 60,
   'f': 0},
  'G': <192x41 sparse matrix of type '<class 'numpy.float64'>'
  	with 1142 stored elements in COOrdinate format>,
  'h': array([-0.        , -0.        , -0.        , -0.        , -0.        ,
         -0.        , -0.        , -0.        , -0.        , -0.        ,
          1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
          1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         -0.        , -0.        , -0.        , 

In [135]:
B.value

array([-0.005     ,  0.005     , -0.005     ,  0.00194372,  0.005     ,
        0.005     ,  0.005     ,  0.005     ,  0.005     ,  0.005     ])

In [10]:
'''
Dual of relaxation: Case 1.
'''
import cvxpy as cp
import numpy as np

n, p = X.shape

alpha = cp.Variable(n)
gamma = cp.Variable(p)
eta = cp.Variable(p)


cons_box = [gamma >= 0, eta >= 0]
constraints = cons_box

# Build the objective

temp_exp = cp.sum(cp.pos(cp.square(X.T * alpha + gamma - eta)/(4*lambda_2)-lambda_0))
obj = cp.Maximize(-0.5*cp.sum_squares(alpha) - temp_exp - alpha * y - M*cp.sum(gamma) - M*cp.sum(eta))
print("Built obj")

# Define the problem
prob = cp.Problem(obj, constraints)
print("Defined problem")
result = prob.solve(solver=cp.MOSEK, verbose=True)

Built obj
Defined problem


Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 172             
  Cones                  : 11              
  Scalar variables       : 273             
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 172             
  Cones                  : 11              
  Scalar variables       : 273             
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer  - threads                : 4               
Optimizer  - solved problem         : the primal      
Optimizer  - Constraints            : 31
Optimizer  - Cones                  : 11
Opt

In [128]:
gamma[4].value

25.307306114858573

In [127]:
i = 4
print(- 2*M*lambda_2 - np.dot(alpha.value, X[:,i]))

25.30729922585898


In [124]:
i = 0
print(eta[0].value)
print(- 2*M*lambda_2 + np.dot(alpha.value, X[:,i]))

7.266161742171839
7.266154051976771


In [136]:
'''
Dual of relaxation: Case 2.
'''
import cvxpy as cp
import numpy as np

n, p = X.shape

alpha = cp.Variable(n)
gamma = cp.Variable(p)
eta = cp.Variable(p)


cons_box = [gamma >= 0, eta >= 0]
cons_linf = [cp.abs(X.T * alpha + gamma - eta) <= lambda_0/M + lambda_2*M]
constraints = cons_box + cons_linf

# Build the objective
obj = cp.Maximize(-0.5*cp.sum_squares(alpha) - alpha * y - M*cp.sum(gamma) - M*cp.sum(eta))
print("Built obj")

# Define the problem
prob = cp.Problem(obj, constraints)
print("Defined problem")
result = prob.solve(solver=cp.MOSEK, verbose=True)

Built obj
Defined problem


Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 152             
  Cones                  : 1               
  Scalar variables       : 233             
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 152             
  Cones                  : 1               
  Scalar variables       : 233             
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer  - threads                : 24              
Optimizer  - solved problem         : the primal      
Optimizer  - Constraints            : 11
Optimizer  - Cones                  : 1
Opti

In [141]:
gamma.value

array([1.20032633e-05, 6.91203600e+00, 1.16255788e-05, 3.10476486e-05,
       5.26653293e+01, 2.05705379e+01, 7.84242940e+01, 1.44898555e+01,
       8.45618297e+00, 2.05052955e+01])

In [146]:
eta.value

array([3.10345800e+01, 8.43463285e-06, 2.07662774e+01, 2.56855232e-05,
       1.21972257e-05, 1.15319860e-05, 1.23356050e-05, 1.09149841e-05,
       9.21753623e-06, 1.16528510e-05])

In [148]:
# gamma
i = 1
- np.dot(alpha.value, X[:,i]) - lambda_0/M + lambda_2*M

6.9130193615865245

In [145]:
# eta
i=0
np.dot(alpha.value, X[:,i]) - lambda_0/M - lambda_2*M

31.034558364736455

In [50]:
np.sqrt(0.001/0.1)

0.1

In [5]:
result.sol

63.13166166695671